# Text generation with an RNN

### Learning Objectives

- Learn how to generate text using a RNN
- Create training examples and targets for text generation
- Build a RNN model for sequence generation using Keras Subclassing
- Create a text generator and evaluate the output

This tutorial demonstrates how to generate text using a character-based RNN. You will work with a dataset of Shakespeare's writing from Andrej Karpathy's [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). Given a sequence of characters from this data ("Shakespear"), train a model to predict the next character in the sequence ("e"). Longer sequences of text can be generated by calling the model repeatedly.

Below is the sample output when the model in this tutorial trained for 30 epochs, and started with the prompt "Q":

<pre>
QUEENE:
I had thought thou hadst a Roman; for the oracle,
Thus by All bids the man against the word,
Which are so weak of care, by old care done;
Your children were in your holy love,
And the precipitation through the bleeding throne.

BISHOP OF ELY:
Marry, and will, my lord, to weep in such a one were prettiest;
Yet now I was adopted heir
Of the world's lamentable day,
To watch the next way with his father with his face?

ESCALUS:
The cause why then we are all resolved more sons.

VOLUMNIA:
O, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, it is no sin it should be dead,
And love and pale as any will to that word.

QUEEN ELIZABETH:
But how long have I heard the soul for this world,
And show his hands of life be proved to stand.

PETRUCHIO:
I say he look'd on, if I must be content
To stay him from the fatal of our country's bliss.
His lordship pluck'd from this sentence then for prey,
And then let us twain, being the moon,
were she such a case as fills m
</pre>

While some of the sentences are grammatical, most do not make sense. The model has not learned the meaning of words, but here are some things to consider:

* The model is character-based. When training started, the model did not know how to spell an English word, or that words were even a unit of text.

* The structure of the output resembles a play—blocks of text generally begin with a speaker name, in all capital letters similar to the dataset.

* As demonstrated below, the model is trained on small batches of text (100 characters each), and is still able to generate a longer sequence of text with coherent structure.

## Installing Tensorflow with GPU Support for Fast Training

To install TensorFlow with GPU support in an Anaconda environment in 2024, follow these steps:

### Create and Activate Anaconda Environment

First, create a new Anaconda environment and activate it by going to the Anaconda Prompt and running the following commands:

```bash
conda create --name tf_gpu python=3.9
conda activate tf_gpu
```

### Install CUDA and cuDNN

Install the CUDA Toolkit and cuDNN library using Conda:

```bash
conda install -c conda-forge cudatoolkit=11.2 cudnn=8.1.0
```

### Install TensorFlow

Upgrade pip and install TensorFlow then downgrade the Numpy version to be compatible:

```bash
pip install --upgrade pip
pip install "tensorflow<2.11"
pip install "numpy<2"
```

*Note: As of 2024, versions above 2.10 are not officially supported for GPU on Windows native installations.*

### Verify the Installation

To verify that TensorFlow can access your GPU:

```bash
python -c "import tensorflow as tf; print(tf.config.list_physical_devices('GPU'))"
```

If a list of GPU devices is returned, you've successfully installed TensorFlow with GPU support.


In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
print("TensorFlow version:", tf.__version__)
print("Is built with CUDA:", tf.test.is_built_with_cuda())
print("GPUs visible to TensorFlow:", tf.config.list_physical_devices('GPU'))

## Setup

### Import TensorFlow and other libraries

In [ ]:
import os  # Importing the `os` module to interact with the operating system
import warnings  # Importing the `warnings` module to manage warning messages
import time  # Importing the `time` module to work with time-related functions

import numpy as np  # Importing `numpy`, a package for numerical computations
import tensorflow as tf  # Importing `tensorflow`, a library for machine learning


In [ ]:
# Suppress all warnings
warnings.filterwarnings("ignore")

# Suppress TensorFlow-specific logs to minimize console output
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

### Download the Shakespeare dataset

Change the following line to run this code on your own data.

In [ ]:
# Using TensorFlow's utility function to download a file and return the path to the file
path_to_file = tf.keras.utils.get_file(
    "shakespeare.txt",  # The name to save the file as locally
    "https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt"  # The URL from which to download the file
)


### Read the data

First, we'll download the file and then decode.

In [ ]:
# Open the file in binary mode, read its contents, and decode it to a UTF-8 string
text = open(path_to_file, "rb").read().decode(encoding="utf-8")

# Print the length of the text in characters
print(f"Length of text: {len(text)} characters")


Let's take a look at the first 250 characters in text

In [ ]:
# Print the first 250 characters of the text to provide a sample of the content
print(text[:250])


Let's check to see how many unique characters are in our corpus/document.

In [ ]:
# Create a sorted list of unique characters found in the text
vocab = sorted(set(text))

# Print the number of unique characters in the text
print(f"{len(vocab)} unique characters")

# Show all the characters in the vocabulary
print(vocab)


## Process the text

### Vectorize the text

Before training, you need to convert the strings to a numerical representation. 

Using `tf.keras.layers.StringLookup` layer can convert each character into a numeric ID. It just needs the text to be split into tokens first.

In [ ]:
# Split the text into individual characters, considering UTF-8 encoding
chars = tf.strings.unicode_split(text, input_encoding="UTF-8")

# Display the resulting tensor of characters
chars


Now create the `tf.keras.layers.StringLookup` layer:

In [ ]:
# Create a layer that maps characters to an integer based on the vocabulary
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab),  # The list of unique characters in the vocabulary
    mask_token=None  # No mask token is used, meaning all characters are treated as valid
)


It converts from tokens to character IDs:

In [ ]:
# Convert the characters to their corresponding integer IDs using the StringLookup layer
ids = ids_from_chars(chars)

# Display the resulting tensor of integer IDs
ids


Since the goal of this tutorial is to generate text, it will also be important to invert this representation and recover human-readable strings from it. For this you can use `tf.keras.layers.StringLookup(..., invert=True)`.  

Note: Here instead of passing the original vocabulary generated with `sorted(set(text))` use the `get_vocabulary()` method of the `tf.keras.layers.StringLookup` layer so that the `[UNK]` tokens is set the same way.

In [ ]:
# Create a layer that maps integer IDs back to their corresponding characters
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(),  # Get the vocabulary from the existing StringLookup layer
    invert=True,  # Invert the mapping to go from IDs back to characters
    mask_token=None  # No mask token is used, meaning all IDs are treated as valid
)


This layer recovers the characters from the vectors of IDs, and returns them as a `tf.RaggedTensor` of characters:

In [ ]:
# Convert the integer IDs back to their corresponding characters using the StringLookup layer
chars = chars_from_ids(ids)

# Display the resulting tensor of characters
chars


You can `tf.strings.reduce_join` to join the characters back into strings. 

In [ ]:
# Join the characters along the last axis (-1) to form complete strings, and convert the result to a NumPy array
result = tf.strings.reduce_join(chars, axis=-1).numpy()

# Display the resulting NumPy array of joined strings
result


Now let's make a function to encapsulate this functionality

In [ ]:
def text_from_ids(ids):
    """
    Convert a sequence of integer IDs back into a string.

    Args:
        ids: A tensor of integer IDs representing characters.

    Returns:
        A TensorFlow string tensor where the IDs have been converted back to a string.
    """
    # Join the characters along the last axis to form a complete string
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)


### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task you're training the model to perform. The input to the model will be a sequence of characters, and you train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create Training Examples and Targets

In this step, we'll break down our text data into sequences that can be used to train the model.

#### Understanding Input and Target Sequences

We'll start by dividing the text into smaller sequences, where each sequence has a fixed number of characters defined by `seq_length`. These sequences are called **input sequences**.

For each input sequence, we need a corresponding **target sequence**. The target sequence is just the input sequence shifted by one character to the right. This means the model will learn to predict the next character in the sequence.

For example, if `seq_length` is set to 4 and our text is "Hello":
- The **input sequence** would be "Hell"
- The **target sequence** would be "ello"

#### Why Add 1 to `seq_length`?

To generate both the input and target sequences from the text, we'll actually create sequences of length `seq_length + 1`. This gives us one extra character at the end of each sequence that becomes the target character when the sequence is split.

So, if `seq_length` is 4:
- The sequence generated from "Hello" will be "Hello" (5 characters).
- We'll split this sequence into:
  - **Input:** "Hell"
  - **Target:** "ello"

#### Converting Text to Character Indices

Before we can create these sequences, we need to convert our text into numerical data that TensorFlow can work with. Each character in the text will be mapped to a unique integer ID (this mapping is typically based on a predefined vocabulary).

We use the `tf.data.Dataset.from_tensor_slices` function to transform the entire text into a stream of these integer IDs. This function takes a list (or array) of data (in this case, the character indices) and creates a TensorFlow dataset where each element corresponds to a single character's ID.

This stream of character IDs will then be divided into sequences of `seq_length + 1`, from which we'll extract our input and target sequences for training.


In [ ]:
# Convert the entire Shakespeare text into a sequence of integer IDs by first splitting it into characters
all_ids = ids_from_chars(tf.strings.unicode_split(text, "UTF-8"))

# Display the resulting tensor of integer IDs representing the entire text
all_ids


In [ ]:
# Create a TensorFlow dataset from the tensor of integer IDs
# The resulting dataset `ids_dataset` now contains each integer ID as a separate element
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [ ]:
# Iterate over the first 5 elements of the `ids_dataset`
for ids in ids_dataset.take(5):
    # Convert each integer ID back to its corresponding character, then decode it to a UTF-8 string
    print(chars_from_ids(ids).numpy().decode("utf-8"))



#### Define the Length of Each Sequence of Characters
```python
seq_length = 100
```
- In this line, we define `seq_length`, which determines the number of characters in each input sequence. 
- This value controls how much context the model will use when learning to predict the next character.
- For example, if `seq_length` is 100, each input sequence fed into the model will contain 100 characters from the text.

#### Calculate the Number of Training Examples per Epoch
```python
examples_per_epoch = len(text) // (seq_length + 1)
```
- Here, we calculate the number of training examples that can be generated from the entire text for one epoch of training.
- The total number of examples is found by dividing the length of the text (`len(text)`) by `seq_length + 1`. 
- The `+1` accounts for the fact that each sequence of `seq_length` characters needs one additional character to create the corresponding target sequence.
- The result, `examples_per_epoch`, tells us how many input-target pairs can be created from the text data. This value helps in understanding the training process and setting up the training loop correctly.


In [ ]:
# Define the length of each sequence of characters
seq_length = 100

# Calculate the number of training examples per epoch
# The total number of examples is determined by dividing the length of the text by the sequence length plus one
examples_per_epoch = len(text) // (seq_length + 1)


The `batch` method lets you easily convert these individual characters to sequences of the desired size.

In [ ]:
# Batch the dataset into sequences of `seq_length + 1` characters
# The `drop_remainder=True` ensures that only full batches are kept
sequences = ids_dataset.batch(seq_length + 1, drop_remainder=True)

# Iterate over the first batch of sequences
for seq in sequences.take(1):
    # Convert the sequence of IDs back into characters and print the result
    print(chars_from_ids(seq))


It's easier to see what this is doing if you join the tokens back into strings:

In [ ]:
# Iterate over the first 5 batches of sequences
for seq in sequences.take(5):
    # Convert each sequence of IDs back into text and print the resulting string
    print(text_from_ids(seq).numpy())


For training you'll need a dataset of `(input, label)` pairs. Where `input` and 
`label` are sequences. At each time step the input is the current character and the label is the next character. 

Here's a function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep:

In [ ]:
def split_input_target(sequence):
    """
    Split a sequence into input and target sequences for training.

    Args:
        sequence: A sequence of text (or IDs) to be split.

    Returns:
        A tuple containing:
            - input_text: The input sequence, which is the original sequence minus the last character.
            - target_text: The target sequence, which is the original sequence minus the first character.
    """
    input_text = sequence[:-1]  # All characters except the last one
    target_text = sequence[1:]  # All characters except the first one
    return input_text, target_text


In [ ]:
# Example usage of the split_input_target function
input_text, target_text = split_input_target(list("Tensorflow"))

# Display the input and target sequences
print("Input:", input_text)
print("Target:", target_text)


In [ ]:
# Apply the split_input_target function to each sequence in the dataset
# The `dataset` now contains pairs of input and target sequences for each original sequence
dataset = sequences.map(split_input_target)


In [ ]:
# Iterate over the first pair of input and target sequences in the dataset
for input_example, target_example in dataset.take(1):
    # Convert the input sequence of IDs back into text and print it
    print("Input :", text_from_ids(input_example).numpy())
    # Convert the target sequence of IDs back into text and print it
    print("Target:", text_from_ids(target_example).numpy())


### Create training batches

You used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, you need to shuffle the data and pack it into batches.

In [ ]:
# Batch size: the number of sequences per batch
BATCH_SIZE = 64

# Buffer size for shuffling the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

# Shuffle the dataset with the specified buffer size,
# then batch the dataset and prefetch to optimize the data pipeline
dataset = (
    dataset.shuffle(BUFFER_SIZE)  # Shuffle the dataset with the buffer size
    .batch(BATCH_SIZE, drop_remainder=True)  # Batch the dataset and drop the last batch if it's incomplete
    .prefetch(tf.data.experimental.AUTOTUNE)  # Prefetch to allow the data pipeline to be optimized by TensorFlow
)

dataset  # The final prepared dataset ready for training


## Build The Model

This section defines the model as a `keras.Model` subclass (For details see [Making new Layers and Models via subclassing](https://www.tensorflow.org/guide/keras/custom_layers_and_models)). 

#### Build a model with the following layers

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map each character-ID to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use an LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

In [ ]:
# Length of the vocabulary in characters
vocab_size = len(vocab)

# The dimension of the embedding vector, which represents each character in a continuous vector space
embedding_dim = 256

# The number of units in the RNN layer, which controls the capacity of the model
rnn_units = 1024


The class below does the following:
- We derive a class from tf.keras.Model
- The constructor is used to define the layers of the model
- We define the pass forward using the layers defined in the constructor

In [ ]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        """
        Initialize the model with an embedding layer, GRU layer, and dense output layer.

        Args:
            vocab_size: The size of the vocabulary (number of unique characters).
            embedding_dim: The dimension of the embedding vector.
            rnn_units: The number of units in the GRU layer.
        """
        super().__init__()
        # Embedding layer: maps integer inputs to dense vectors of a fixed size
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        
        # GRU layer: a type of RNN layer that is particularly good for sequential data
        self.gru = tf.keras.layers.GRU(
            rnn_units,
            return_sequences=True,  # Return the full sequence of outputs
            return_state=True,  # Return the final state in addition to the output
            recurrent_initializer='glorot_uniform'  # Initialize the recurrent weights
        )
        
        # Dense layer: outputs predictions for the next character in the sequence
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        """
        Run the forward pass of the model.

        Args:
            inputs: The input data, typically sequences of integers representing characters.
            states: The initial state of the RNN (optional).
            return_state: Whether to return the final state (optional).
            training: Whether the model is in training mode (optional).

        Returns:
            If return_state is True, returns both the output and the final state.
            Otherwise, returns just the output.
        """
        # Pass inputs through the embedding layer
        x = self.embedding(inputs, training=training)
        
        # Handle the initial state explicitly if not provided
        if states is None:
            states = tf.zeros([tf.shape(x)[0], self.gru.units])
        
        # Pass the embedded inputs and initial states through the GRU layer
        x, states = self.gru(x, initial_state=states, training=training)
        
        # Pass the output of the GRU layer through the dense layer
        x = self.dense(x, training=training)

        # Return the output and optionally the state
        if return_state:
            return x, states
        else:
            return x


In [ ]:
# Instantiate the model with the specified vocabulary size, embedding dimension, and number of RNN units
model = MyModel(
    vocab_size=len(ids_from_chars.get_vocabulary()),  # Determine the vocabulary size based on the StringLookup layer
    embedding_dim=embedding_dim,  # The dimension of the embedding vector
    rnn_units=rnn_units,  # The number of units in the GRU layer
)


In [ ]:
# Compile the model with the Adam optimizer and sparse categorical cross-entropy loss function
model.compile(
    optimizer='adam',  # Adam optimizer is used for training the model
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True)  # Loss function that handles integer labels directly
)


For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character.

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [ ]:
# Example usage of the model with a batch of data
for input_example_batch, target_example_batch in dataset.take(1):
    # Generate predictions for the input batch using the model
    example_batch_predictions = model(input_example_batch)
    
    # Print the shape of the predictions tensor
    # The shape should be (batch_size, sequence_length, vocab_size)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length.

#### Model Summary

The output below provides a summary of the neural network model architecture, which is generated by calling `model.summary()`. This summary includes the following key pieces of information:

1. **Model Name**: 
   - The name of the model is displayed at the top. In this case, the model is named `"my_model"`.

2. **Layer Information**:
   - **Layer (type)**: This column lists each layer in the model, along with its type. For instance, we have an `Embedding` layer, a `GRU` (Gated Recurrent Unit) layer, and a `Dense` (fully connected) layer.
   - **Output Shape**: This column provides the shape of the output produced by each layer. The term "multiple" indicates that the output shape may vary depending on the input size.
   - **Param #**: This column shows the number of trainable parameters in each layer. These parameters are the weights that the model will learn during training.

3. **Total Parameters**:
   - **Total params**: The total number of parameters in the model, which is the sum of parameters in all layers. For this model, there are 4,022,850 parameters.
   - **Trainable params**: The total number of parameters that will be updated during training, which is the same as the total parameters in this case, indicating that all parameters are trainable.
   - **Non-trainable params**: The number of parameters that are not trainable. Here, it's zero, meaning there are no parameters that are frozen or fixed during training.

This summary is helpful for understanding the complexity of the model, the number of parameters involved, and how the data flows through each layer.


In [ ]:
# Display a summary of the model architecture
model.summary()


To get actual predictions from the model you need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [ ]:
# Sample indices from the first sequence's predicted probabilities using categorical sampling
sampled_indices = tf.random.categorical(
    example_batch_predictions[0],  # Use the predictions from the first sequence in the batch
    num_samples=1  # Sample one index for each time step in the sequence
)

# Remove the extra dimension from the sampled indices and convert to a NumPy array
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()


This gives us, at each timestep, a prediction of the next character index:

In [ ]:
# Display the sampled indices
sampled_indices


#### 2. Decoding the Predicted Text
- **Input Sequence:** 
  - The first print statement converts the `input_example_batch[0]` back into readable text using the `text_from_ids` function. This shows the actual input sequence that the model was given.
  - **Explanation:** This is the sequence of characters that the model was supposed to use to predict the next character at each timestep.

- **Predicted Sequence:** 
  - The second print statement converts the `sampled_indices` into readable text, showing the sequence of characters that the model predicted.
  - **Explanation:** This is what the model predicts will follow the input sequence. Since the model is untrained or undertrained, the predictions are mostly nonsensical, containing a mix of characters that don’t form coherent words or sentences.

### Understanding the Output

- **Input:** The text that was fed into the model. For example, "`b't what you swear!\\nLook, as I blow this feather from my face,\\nAnd as the air blows it to me again,\\nOb'`" is a coherent sequence, probably taken from some literary text.

- **Next Char Predictions:** This is the model's attempt to predict the following characters based on the input. Since this is an untrained or poorly trained model, the predictions "`b'!pzTD$QgfD&DFfQjDGbDuvK\\n3deS?HIVwrPHzwMgc?;TgPBYoVqVzaNBA\\n!vl?rkO!jCZU,\\nus:nZjReGYSKr;nKFYMmqzn3oDiWj'`" are mostly random and don’t form meaningful text.

### What This Means

- **Current Model Performance:** The model is currently not performing well because it has either not been trained or is in the very early stages of training. The predictions are essentially random, showing that the model hasn’t yet learned the patterns in the text data.

- **Next Steps:** As training progresses, you would expect the model's predictions to become more accurate and start forming coherent text that closely matches the patterns in the training data.

This code snippet is part of the process where you can visually inspect how well your model is learning to predict the next character in a sequence. Initially, the predictions will be poor, but with training, you should see improvements.

In [ ]:
# Print the input sequence by converting IDs back to text
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())

# Print a blank line for separation
print()

# Print the predicted next characters based on the sampled indices
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())


## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

#### Attach an optimizer, and a loss function
The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because your model returns logits, you need to set the `from_logits` flag.


In [ ]:

# Define the loss function
# SparseCategoricalCrossentropy is used because the labels are integer indices rather than one-hot encoded
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)


In [ ]:
# Calculate the mean loss for the example batch
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)

# Print the shape of the predictions tensor
print(
    "Prediction shape: ",
    example_batch_predictions.shape,
    " # (batch_size, sequence_length, vocab_size)",
)

# Print the calculated mean loss for the example batch
print("Mean loss:        ", example_batch_mean_loss)


A newly initialized model shouldn't be too sure of itself, the output logits should all have similar magnitudes. To confirm this you can check that the exponential of the mean loss is approximately equal to the vocabulary size. A much higher loss means the model is sure of its wrong answers, and is badly initialized:

In [ ]:
# Calculate the exponential of the mean loss
# This can be interpreted as the perplexity, a measure of how well the model is predicting the next character
perplexity = tf.exp(example_batch_mean_loss).numpy()

# Display the perplexity value
perplexity


### Compiling the Model

In this step, we are configuring the model for training by compiling it. The `model.compile()` function is where we specify how the model will be optimized during training, and what loss function it will use to measure its performance.

#### Code Breakdown

- **`optimizer="adam"`**:
  - **Adam Optimizer**: 
    - The `Adam` optimizer is a popular choice for training neural networks because it combines the benefits of two other optimizers: AdaGrad (which works well with sparse gradients) and RMSProp (which works well in online and non-stationary settings).
    - It is efficient and requires less memory, making it suitable for large datasets and models. Additionally, Adam adapts the learning rate during training, which helps in faster convergence.
  - **Default Settings**: 
    - In this code, we're using the default settings for the Adam optimizer. These default settings include parameters like learning rate, beta1, beta2, and epsilon, which control how the optimizer updates the model's weights during training.

- **`loss=loss`**:
  - **Loss Function**:
    - The `loss` parameter specifies the loss function that will be used to calculate how well the model's predictions match the target values. In this case, the loss function has been defined earlier in the code as `SparseCategoricalCrossentropy`.
    - **SparseCategoricalCrossentropy**:
      - This loss function is typically used for multi-class classification problems where each class is represented by an integer (like in character-level text generation). It computes the cross-entropy loss between the true labels and the predicted labels, but it’s optimized for the case where labels are sparse (i.e., they are integers rather than one-hot encoded vectors).

#### Why This Matters

- **Optimization**:
  - Compiling the model is a crucial step because it defines how the model will learn from the data. The optimizer and loss function play a key role in determining the model's ability to converge on a solution that minimizes errors and makes accurate predictions.

- **Flexibility**:
  - By choosing Adam and SparseCategoricalCrossentropy, we are ensuring that the model has a robust and flexible training setup that is well-suited for the type of problem we are solving (e.g., text generation, language modeling).

This compilation step sets the stage for training the model, where it will iteratively adjust its internal parameters (weights) to minimize the loss and improve its predictions.


In [ ]:
# Compile the model with the Adam optimizer and the previously defined loss function
model.compile(
    optimizer="adam",  # Adam optimizer is used for its efficiency and adaptive learning rate
    loss=loss  # The loss function is the SparseCategoricalCrossentropy defined earlier
)


### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = "./training_checkpoints"

# Name of the checkpoint files, including the directory and the epoch number in the filename
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

# Define a callback to save the model's weights during training
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,  # Path where the checkpoint files will be saved
    save_weights_only=True  # Only the model's weights will be saved, not the entire model
)


### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [ ]:
# Define the number of epochs for training the model
EPOCHS = 10

# Train the model using the dataset for the specified number of epochs
# The checkpoint_callback is used to save the model's weights after each epoch
history = model.fit(
    dataset,  # The dataset to train on
    epochs=EPOCHS,  # The number of complete passes through the dataset
    callbacks=[checkpoint_callback]  # List of callbacks to apply during training, including saving checkpoints
)

#### Generate text

The simplest way to generate text with this model is to run it in a loop, and keep track of the model's internal state as you execute it.

Each time you call the model you pass in some text and an internal state. The model returns a prediction for the next character and its new state. Pass the prediction and state back in to continue generating text.


The following makes a single step prediction:

In [ ]:
class OneStep(tf.keras.Model):
    def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
        """
        Initialize the OneStep model for generating text one character at a time.

        Args:
            model: The trained model used for generating text.
            chars_from_ids: A function that converts token IDs to characters.
            ids_from_chars: A function that converts characters to token IDs.
            temperature: Controls the randomness of predictions by scaling the logits.
        """
        super().__init__()
        self.temperature = temperature  # Set the temperature for controlling prediction randomness
        self.model = model  # The trained character-level model
        self.chars_from_ids = chars_from_ids  # Function to convert IDs to characters
        self.ids_from_chars = ids_from_chars  # Function to convert characters to IDs

        # Create a mask to prevent the "[UNK]" token from being generated
        skip_ids = self.ids_from_chars(["[UNK]"])[:, None]
        sparse_mask = tf.SparseTensor(
            values=[-float("inf")] * len(skip_ids),  # Assign -inf to bad indices
            indices=skip_ids,  # Indices of the tokens to be masked
            dense_shape=[len(ids_from_chars.get_vocabulary())]  # Match the shape to the vocabulary size
        )
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)

    @tf.function
    def generate_one_step(self, inputs, states=None):
        """
        Generate a single step (character) in the sequence.

        Args:
            inputs: The input characters to the model.
            states: The initial states for the RNN.
        
        Returns:
            predicted_chars: The next predicted character.
            states: The updated states of the RNN.
        """
        # Convert input strings to token IDs
        input_chars = tf.strings.unicode_split(inputs, "UTF-8")
        input_ids = self.ids_from_chars(input_chars).to_tensor()

        # Run the model to get the predicted logits and updated states
        predicted_logits, states = self.model(
            inputs=input_ids, states=states, return_state=True
        )

        # Use only the last prediction in the sequence
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits / self.temperature  # Adjust logits by temperature

        # Apply the prediction mask to prevent "[UNK]" from being generated
        predicted_logits = predicted_logits + self.prediction_mask

        # Sample from the adjusted logits to get the next token ID
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)

        # Convert the token IDs back to characters
        predicted_chars = self.chars_from_ids(predicted_ids)

        # Return the predicted character and the updated states
        return predicted_chars, states


In [ ]:
# Instantiate the OneStep model for generating text one character at a time
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)


Run it in a loop to generate some text. Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [ ]:
# Start the timer to measure the runtime of text generation
start = time.time()

# Initialize the RNN states to None for the first prediction
states = None

# Set the starting string for text generation
next_char = tf.constant(["ROMEO:"])

# Initialize a list to hold the generated characters, starting with the input string
result = [next_char]

# Generate 1000 characters one at a time
for n in range(1000):
    # Generate the next character and update the RNN states
    next_char, states = one_step_model.generate_one_step(
        next_char, states=states
    )
    # Append the generated character to the result list
    result.append(next_char)

# Join the list of strings into a single string
result = tf.strings.join(result)

# Stop the timer to calculate runtime
end = time.time()

# Print the generated text and the runtime
print(result[0].numpy().decode("utf-8"), "\n\n" + "_" * 80)
print("\nRun time:", end - start)


The easiest thing you can do to improve the results is to train it for longer (try `EPOCHS = 30`).

You can also experiment with a different start string, try adding another RNN layer to improve the model's accuracy, or adjust the temperature parameter to generate more or less random predictions.

If you want the model to generate text *faster* the easiest thing you can do is batch the text generation. In the example below the model generates 5 outputs in about the same time it took to generate 1 above. 

In [ ]:
# Start the timer to measure the runtime of text generation
start = time.time()

# Initialize the RNN states to None for the first prediction
states = None

# Set the starting strings for text generation with multiple instances of "ROMEO:"
next_char = tf.constant(["ROMEO:", "ROMEO:", "ROMEO:", "ROMEO:", "ROMEO:"])

# Initialize a list to hold the generated characters, starting with the input strings
result = [next_char]

# Generate 1000 characters one at a time for each starting string
for n in range(1000):
    # Generate the next character and update the RNN states
    next_char, states = one_step_model.generate_one_step(
        next_char, states=states
    )
    # Append the generated characters to the result list
    result.append(next_char)

# Join the list of strings into a single tensor
result = tf.strings.join(result)

# Stop the timer to calculate runtime
end = time.time()

# Print the generated text for all starting strings and the runtime
print(result, "\n\n" + "_" * 80)
print("\nRun time:", end - start)


Clean up the output

In [ ]:
import textwrap

# Function to format the text for better readability
def format_generated_text(text_tensor):
    # Convert the tensor to string
    text = text_tensor.numpy().decode('utf-8')
    
    # Split into lines based on '\n'
    lines = text.split('\n')
    
    # Format each line
    formatted_lines = []
    for line in lines:
        if line.strip():  # Only process non-empty lines
            # Check if the line starts with a character name (simple heuristic)
            if ":" in line:
                name, dialogue = line.split(":", 1)
                formatted_line = f"{name.strip()}:{textwrap.fill(dialogue.strip(), width=70, subsequent_indent=' ' * 4)}"
            else:
                formatted_line = textwrap.fill(line.strip(), width=70)
            formatted_lines.append(formatted_line)
        else:
            formatted_lines.append("")
    
    # Join formatted lines back into a single string
    formatted_text = "\n\n".join(formatted_lines)
    
    return formatted_text

# Apply the formatting to each generated result and print it
for i in range(result.shape[0]):
    formatted_text = format_generated_text(result[i])
    print(formatted_text)
    print("\n" + "_" * 80)


## Export the generator

This single-step model can easily be [saved and restored](https://www.tensorflow.org/guide/saved_model), allowing you to use it anywhere a `tf.saved_model` is accepted.

In [ ]:
# Save the OneStep model to the specified directory
tf.saved_model.save(one_step_model, "one_step")

# Reload the saved model from the specified directory
one_step_reloaded = tf.saved_model.load("one_step")


In [ ]:
# Initialize the RNN states to None for the first prediction
states = None

# Set the starting string for text generation
next_char = tf.constant(["ROMEO:"])

# Initialize a list to hold the generated characters, starting with the input string
result = [next_char]

# Generate 100 characters one at a time using the reloaded model
for n in range(100):
    # Generate the next character and update the RNN states
    next_char, states = one_step_reloaded.generate_one_step(
        next_char, states=states
    )
    # Append the generated character to the result list
    result.append(next_char)

# Join the list of strings into a single string
final_result = tf.strings.join(result)

# Print the generated text after decoding it from a tensor to a UTF-8 string
print(final_result[0].numpy().decode("utf-8"))


## Advanced: Customized Training

The above training procedure is simple, but does not give you much control.
It uses teacher-forcing which prevents bad predictions from being fed back to the model, so the model never learns to recover from mistakes.

So now that you've seen how to run the model manually next you'll implement the training loop. This gives a starting point if, for example, you want to implement _curriculum  learning_ to help stabilize the model's open-loop output.

The most important part of a custom training loop is the train step function.

Use `tf.GradientTape` to track the gradients. You can learn more about this approach by reading the [eager execution guide](https://www.tensorflow.org/guide/eager).

The basic procedure is:

1. Execute the model and calculate the loss under a `tf.GradientTape`.
2. Calculate the updates and apply them to the model using the optimizer.

In [ ]:
class CustomTraining(MyModel):
    @tf.function
    def train_step(self, inputs):
        """
        Perform a single training step.

        Args:
            inputs: A tuple containing the input data and the corresponding labels.

        Returns:
            A dictionary containing the loss value for the training step.
        """
        # Unpack the input data and labels
        inputs, labels = inputs
        
        # Record the operations for automatic differentiation
        with tf.GradientTape() as tape:
            # Make predictions using the model
            predictions = self(inputs, training=True)
            # Calculate the loss using the model's loss function
            loss = self.loss(labels, predictions)
        
        # Compute the gradients of the loss with respect to the model's trainable variables
        grads = tape.gradient(loss, self.trainable_variables)
        
        # Apply the gradients to the optimizer to update the model's weights
        self.optimizer.apply_gradients(zip(grads, self.trainable_variables))

        # Return the loss value as a dictionary
        return {"loss": loss}


The above implementation of the `train_step` method follows [Keras' `train_step` conventions](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit). This is optional, but it allows you to change the behavior of the train step and still use keras' `Model.compile` and `Model.fit` methods.

In [ ]:
# Instantiate the CustomTraining model with the specified parameters
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),  # Determine the vocabulary size based on the StringLookup layer
    embedding_dim=embedding_dim,  # The dimension of the embedding vector
    rnn_units=rnn_units  # The number of units in the GRU layer
)


In [ ]:
# Compile the CustomTraining model with the Adam optimizer and sparse categorical cross-entropy loss function
model.compile(
    optimizer=tf.keras.optimizers.Adam(),  # Adam optimizer is used for training the model
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),  # Loss function for integer labels, expecting logits as inputs
)


In [ ]:
# Train the CustomTraining model on the dataset for 1 epoch
model.fit(
    dataset,  # The dataset to train on
    epochs=1  # Number of epochs to train the model
)


Or if you need more control, you can write your own complete custom training loop:

In [ ]:
EPOCHS = 10  # Number of epochs to train the model

# Create a Mean metric to track the average loss during training
mean = tf.metrics.Mean()

# Loop over each epoch
for epoch in range(EPOCHS):
    start = time.time()  # Start timing the epoch

    mean.reset_states()  # Reset the Mean metric at the start of each epoch
    for batch_n, (inp, target) in enumerate(dataset):
        # Perform a training step and get the logs (including the loss)
        logs = model.train_step([inp, target])
        # Update the Mean metric with the loss from the current batch
        mean.update_state(logs["loss"])

        # Print the loss every 50 batches
        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # Save the model's weights as a checkpoint every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    # Print the average loss for the epoch and the time taken
    print()
    print(f"Epoch {epoch+1} Loss: {mean.result().numpy():.4f}")
    print(f"Time taken for 1 epoch {time.time() - start:.2f} sec")
    print("_" * 80)

# Save the model's weights after the final epoch
model.save_weights(checkpoint_prefix.format(epoch=epoch))


## Using the Model

In [ ]:
# Example of generating text after training
start_string = "JULIET:"  # The initial string to start generating text
num_generate = 1000  # Number of characters to generate

# Convert the start string to a TensorFlow constant
input_eval = [start_string]
input_eval = tf.constant(input_eval)

# Initialize a list to hold the generated text
generated_text = []

# Initialize RNN states to None for the first prediction
states = None

# Generate characters one by one
for i in range(num_generate):
    # Generate the next character and update the RNN states
    next_char, states = one_step_model.generate_one_step(input_eval, states=states)
    
    # Use the generated character as the next input to the model
    input_eval = next_char
    
    # Append the generated character to the list
    generated_text.append(next_char)

# Join the list of generated characters into a single string
generated_text = tf.strings.join(generated_text).numpy()[0].decode('utf-8')

# Print the generated text
print(generated_text)



# Summary and Conclusion
In this notebook, we built and trained a character-level language model using TensorFlow. The model was trained on a dataset of text to predict the next character in a sequence, and we demonstrated its ability to generate text by simulating a play-like structure. The notebook also provided a custom training loop, model checkpointing, and text generation examples.

**Next Steps:**
- Experiment with different architectures (e.g., LSTM instead of GRU).
- Fine-tune the model on different datasets or with more sophisticated data preprocessing.
- Deploy the model in a web app to allow interactive text generation.
